## Cull Swap TX
* See [0x90bc3e68ba8393a3bf2d79309365089975341a43](https://polygonscan.com/address/0x90bc3e68ba8393a3bf2d79309365089975341a43) for contract on polygon explorer

In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from datetime import datetime
from pachira import *

In [2]:
POLY_POOL_CONTRACT = '0x90bc3e68ba8393a3bf2d79309365089975341a43'

In [3]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()
start = start - 10000
print(f'latest_block {latest_block}, start {start}, end {end}')

latest_block 65328823, start 65318824, end 65328823


In [6]:
rSwapEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rSwapEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token?'] = Conversion().convert_uint256_hex_string_to_address(topics[0])
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1In'] = Conversion().convert_int256_bytes_to_int(arguments[1]) 
    event['details']['amount0Out'] = Conversion().convert_int256_bytes_to_int(arguments[2]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3]) 
    
    events[k] = event
    if key not in processed_events and event['address'] == POLY_POOL_CONTRACT:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:65,318,833 tx:0x30e814ce90ad10038816dba412c4e2cebd2284ac44b39dd26f25ae0240aa9d1a
Swap at block:65,318,914 tx:0xbc758390225edfe6818db7f1638ca59d9f49f4d715a872dcac926bafcdc6be25
Swap at block:65,318,928 tx:0xfdb0256805078b539dbbde29f49e49852746f1c0ed83d35a4056b429a0fed664
Swap at block:65,318,942 tx:0xf572d63977766a1c68a038920ba9cb880b1bdb1a80067f892dfb9d2547f7814f
Swap at block:65,319,031 tx:0x472d6bc0580b49728366aea2225a6da7c8b08d50ded70e3fe1955f2ec1c5719d
Swap at block:65,319,039 tx:0x429a5c0d3b3000ca7aec75b1f578b4e5152558cdbc163e942f2be920c92dd755
Swap at block:65,319,053 tx:0xc930df33ab4dbc84611527431211d74ea505e55f30bca75d3fdc9864182eab50
Swap at block:65,319,058 tx:0x56792949a06d7ae3a4cb343ce7e8601201f21417ef003bf1acbd5731e6ee7c96
Swap at block:65,319,075 tx:0x55fdc1b284a8afe369863aa3aebe60279104c9255d3cdff708401441fddb1bf6
Swap at block:65,319,141 tx:0x2ea8a83cc87d8c4a8713dde0b9202f95ad6fa34734f94c9d75838cf7b998d60f
Swap at block:65,319,156 tx:0x1329be23d7b489416e97

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events = df_events[df_events['address'].values == POLY_POOL_CONTRACT]
df_events.sort_values(by=["blk_num"], ascending=[False], inplace=True)
df_events.reset_index(drop=True, inplace=True)

In [10]:
swap_events = {}

AAVE_TKN = 'AAVE'
WETH_TKN = 'WETH'

for k in range(len(df_events)):

    token0_name = WETH_TKN if df_events.iloc[k]['details']['amount0In'] > 0 else AAVE_TKN
    token1_name = AAVE_TKN if df_events.iloc[k]['details']['amount1Out'] > 0 else WETH_TKN
    
    swap_events[k] = {}
    swap_events[k]['timestamp'] = df_events.iloc[k]['timestamp']
    swap_events[k]['date'] = datetime.fromtimestamp(swap_events[k]['timestamp'])    
    swap_events[k]['blk_num'] = df_events.iloc[k]['blk_num']
    swap_events[k]['contract'] = df_events.iloc[k]['address']
    swap_events[k]['token0_name'] = token0_name
    swap_events[k]['token1_name'] = token1_name
    swap_events[k]['type'] = "_for_".join([token0_name, token1_name])
    if(df_events.iloc[k]['details']['amount0In'] > 0):
        swap_events[k]['amount0In'] = df_events.iloc[k]['details']['amount0In']
    else:
        swap_events[k]['amount0In'] = df_events.iloc[k]['details']['amount1In']
        
    if(df_events.iloc[k]['details']['amount1Out'] > 0):    
        swap_events[k]['amount1Out'] = df_events.iloc[k]['details']['amount1Out']
    else:
        swap_events[k]['amount1Out'] = df_events.iloc[k]['details']['amount0Out']
        
df_swap_events = pd.DataFrame.from_dict(swap_events, orient='index') 
df_swap_events.sort_values(by=["blk_num"], ascending=[False])

,timestamp,date,blk_num,contract,token0_name,token1_name,type,amount0In,amount1Out
0,1733871028,2024-12-10 14:50:28,65328645,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,134034839027098176,10004999517926150
1,1733870422,2024-12-10 14:40:22,65328360,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,133878597946646880,10004223375417902
2,1733870322,2024-12-10 14:38:42,65328328,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,178343940353639648,13343879261281490
3,1733869556,2024-12-10 14:25:56,65327986,0x90bc3e68ba8393a3bf2d79309365089975341a43,WETH,AAVE,WETH_for_AAVE,10005526704744556,132900863877594312
4,1733869298,2024-12-10 14:21:38,65327865,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,293481052129571881,21976920087587359
...,...,...,...,...,...,...,...,...,...
152,1733850018,2024-12-10 09:00:18,65319031,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,232744804474571546,17005029583964936
153,1733849828,2024-12-10 08:57:08,65318942,0x90bc3e68ba8393a3bf2d79309365089975341a43,WETH,AAVE,WETH_for_AAVE,17231076669584456,234428673371462467
154,1733849800,2024-12-10 08:56:40,65318928,0x90bc3e68ba8393a3bf2d79309365089975341a43,AAVE,WETH,AAVE_for_WETH,163570458178875501,11947387600461779
155,1733849770,2024-12-10 08:56:10,65318914,0x90bc3e68ba8393a3bf2d79309365089975341a43,WETH,AAVE,WETH_for_AAVE,19345910876546196,263383180488094146


### Format
* Formatting according to [batch swapper](https://github.com/SYS-Labs/pachira/blob/feat/batcher-simple/batcher/trades.csv)

In [11]:
token0_nm = WETH_TKN
token1_nm = AAVE_TKN
trades = {}

for event in swap_events:
    trades[event] = {}
    trades[event]['date'] = swap_events[event]['date']
    trades[event]['direction'] = 'token0to1' if swap_events[event]['token0_name'] == token0_nm else 'token1to0'
    trades[event]['amount'] = swap_events[event]['amount0In']

df_trades = pd.DataFrame.from_dict(trades, orient='index') 
df_trades

,date,direction,amount
0,2024-12-10 14:50:28,token1to0,134034839027098176
1,2024-12-10 14:40:22,token1to0,133878597946646880
2,2024-12-10 14:38:42,token1to0,178343940353639648
3,2024-12-10 14:25:56,token0to1,10005526704744556
4,2024-12-10 14:21:38,token1to0,293481052129571881
...,...,...,...
152,2024-12-10 09:00:18,token1to0,232744804474571546
153,2024-12-10 08:57:08,token0to1,17231076669584456
154,2024-12-10 08:56:40,token1to0,163570458178875501
155,2024-12-10 08:56:10,token0to1,19345910876546196


### Dump data

In [18]:
base_dir =  os.getcwd().replace("pachira-python/notebook/cull","")
os.chdir(base_dir)

In [19]:
df_swap_events.to_csv('pachira-python/resources/cull/swap_events.csv', index=False)
df_trades.to_csv('pachira-python/resources/cull/trades.csv', index=False)